In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [2]:
from transformers.trainer_utils import get_last_checkpoint

latest = get_last_checkpoint('mixtral-deepspeed')
latest

'mixtral-deepspeed/checkpoint-2320'

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch

In [4]:
tokenizer = AutoTokenizer.from_pretrained(latest, trust_remote_code=True)

In [5]:
base_model = AutoModelForCausalLM.from_pretrained(
    'mistralai/Mixtral-8x7B-Instruct-v0.1', torch_dtype=torch.bfloat16)

Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

In [6]:
model = PeftModel.from_pretrained(base_model, latest, torch_dtype=torch.bfloat16)
model

/home/ubuntu/.local/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/home/ubuntu/.local/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MixtralForCausalLM(
      (model): MixtralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MixtralDecoderLayer(
            (self_attn): MixtralAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear(
                (base_layer): Linear(in_featur

In [ ]:
model = model.merge_and_unload()

In [ ]:
model.push_to_hub()

In [ ]:
!mkdir malaysian-mixtral-16k-qlora

In [ ]:
model.push_to_hub('malaysian-mixtral-16k-qlora', use_temp_dir = False,
                  organization='mesolitica', safe_serialization=True)

In [ ]:
tokenizer.push_to_hub('malaysian-mixtral-16k-qlora', organization='mesolitica', safe_serialization=True)